In [1]:
# === Multi-horizon AQI with LSTM + Optuna (mirrors your LGBM pipeline) ===
from pathlib import Path
import json
import warnings
warnings.filterwarnings("ignore")

# ----------------- Imports -----------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib
import os

# Torch (install on the fly if needed)
try:
    import torch
    import torch.nn as nn
    from torch.utils.data import DataLoader, TensorDataset
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "torch"])
    import torch
    import torch.nn as nn
    from torch.utils.data import DataLoader, TensorDataset

# Optuna (install if needed)
try:
    import optuna
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "optuna"])
    import optuna

# SHAP (install if needed)
try:
    import shap
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "shap"])
    import shap


In [2]:

# ----------------- CONFIG (kept same style/paths) -----------------
PROJECT_ROOT  = Path.cwd().parent        # notebook inside Model_training/
DATA_PATH     = PROJECT_ROOT / "preprocessed_aqi_data (3).csv"
FEATURES_JSON = PROJECT_ROOT / "final_feature_list.json"

OUT_DIR       = PROJECT_ROOT / "predictions"
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_FILE   = OUT_DIR / "lstm_predicted_aqi_72hrs.csv"      # <— LSTM CSV (no overwrite of LGBM CSV)

WINDOW_SIZE     = 24       # past hours per sample
PREDICT_HORIZON = 72       # next 72 hours (3 days)
TARGET_COL      = "us_aqi"
TIME_COL_CANDS  = ["time", "datetime"]

# Default training params (used for final retrain unless replaced by Optuna best)
EPOCHS          = 40
BATCH_SIZE      = 256
LR              = 1e-3
WD              = 1e-4
PATIENCE        = 8
HIDDEN_SIZE     = 128
NUM_LAYERS      = 2
DROPOUT         = 0.2
BIDIRECTIONAL   = False

RANDOM_SEED     = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"


In [3]:

# ------------------- LOAD & PARSE TIME (DAY-FIRST) -------------------
df = pd.read_csv(DATA_PATH)

# pick time column name automatically
for c in TIME_COL_CANDS:
    if c in df.columns:
        TIME_COL = c
        break
else:
    raise ValueError(f"No datetime column found. Expected one of: {TIME_COL_CANDS}")

# clean & parse as day-first to match your “4/8/25 = 4 Aug 2025”
raw_time = (
    df[TIME_COL].astype(str)
      .str.strip()
      .str.replace("\u00A0", " ", regex=False)
      .str.replace("\u202F", " ", regex=False)
)
df[TIME_COL] = pd.to_datetime(raw_time, dayfirst=True, errors="coerce")

# sort chronologically
df = df.sort_values(TIME_COL).reset_index(drop=True)

# ------------------- FEATURES -------------------
feat_cols = json.loads(FEATURES_JSON.read_text())
missing = [c for c in feat_cols + [TARGET_COL, TIME_COL] if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in data: {missing}")

# ------------------- Helper: build windows (sequence version) -------------------
def build_sequences_limit(frame, features, target, window, horizon):
    n = len(frame)
    return n - window - horizon

def build_sequences_seq(frame, features, target, window, horizon):
    """
    Returns:
      X: (N, T, F) oldest→newest order
      Y: (N, H)
    """
    Xs, Ys = [], []
    Xmat = frame[features].values
    yvec = frame[target].values
    n = len(frame)
    limit = n - window - horizon
    for i in range(limit):
        Xs.append(Xmat[i:i+window])                        # (T, F)
        Ys.append(yvec[i+window:i+window+horizon])         # (H,)
    return np.asarray(Xs), np.asarray(Ys), limit

# ------------------- TRAIN-ONLY WINSORIZATION + IMPUTE + SCALE -------------------
limit = build_sequences_limit(df, feat_cols, TARGET_COL, WINDOW_SIZE, PREDICT_HORIZON)
if limit <= 0:
    raise ValueError("Not enough rows to make sliding windows. Add more data.")

# map “train windows” back to raw rows used by their inputs (same logic as your LGBM)
train_windows = int(limit * 0.8)
raw_end_for_train_inputs = (train_windows - 1) + WINDOW_SIZE
raw_end_for_train_inputs = max(raw_end_for_train_inputs, WINDOW_SIZE)

# Winsorize on TRAIN INPUT rows only
numeric_feats = [c for c in feat_cols if pd.api.types.is_numeric_dtype(df[c])]
low = df.loc[:raw_end_for_train_inputs, numeric_feats].quantile(0.01).to_dict()
high = df.loc[:raw_end_for_train_inputs, numeric_feats].quantile(0.99).to_dict()
for c in numeric_feats:
    df[c] = df[c].clip(lower=low[c], upper=high[c])

# Median impute using TRAIN INPUT rows only, then apply to all
train_medians = df.loc[:raw_end_for_train_inputs, feat_cols].median(numeric_only=True).to_dict()
df[feat_cols] = df[feat_cols].fillna(train_medians)

# Scale features for LSTM (fit on TRAIN INPUT rows only)
scaler = StandardScaler()
scaler.fit(df.loc[:raw_end_for_train_inputs, feat_cols].values)
df[feat_cols] = scaler.transform(df[feat_cols].values)


In [4]:

# ------------------- BUILD SEQUENCES AFTER CLEANING/SCALING -------------------
X_seq, y_seq, limit = build_sequences_seq(df, feat_cols, TARGET_COL, WINDOW_SIZE, PREDICT_HORIZON)
N, T, F = X_seq.shape
H = y_seq.shape[1]
assert T == WINDOW_SIZE and H == PREDICT_HORIZON

# ------------------- TRAIN/VAL SPLIT (chronological 80/20) -------------------
X_train, X_val, y_train, y_val = train_test_split(
    X_seq, y_seq, test_size=0.2, shuffle=False
)

# ------------------- LSTM Model -------------------
class LSTMForecast(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, horizon, bidirectional=False):
        super().__init__()
        self.bidirectional = bidirectional
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True,
            bidirectional=bidirectional
        )
        out_dim = hidden_size * (2 if bidirectional else 1)
        self.head = nn.Sequential(
            nn.Linear(out_dim, 256),
            nn.ReLU(),
            nn.Linear(256, horizon)
        )

    def forward(self, x):               # x: (B, T, F)
        out, _ = self.lstm(x)           # (B, T, hidden*(1|2))
        last = out[:, -1, :]            # last time step
        return self.head(last)          # (B, horizon)

# ------------------- Training helpers -------------------
def make_loaders(Xtr, Ytr, Xva, Yva, bs):
    train_dl = DataLoader(TensorDataset(
        torch.tensor(Xtr, dtype=torch.float32),
        torch.tensor(Ytr, dtype=torch.float32)
    ), batch_size=bs, shuffle=False)
    val_dl = DataLoader(TensorDataset(
        torch.tensor(Xva, dtype=torch.float32),
        torch.tensor(Yva, dtype=torch.float32)
    ), batch_size=bs, shuffle=False)
    return train_dl, val_dl

def eval_loader(model, dl):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xb, yb in dl:
            xb = xb.to(device); yb = yb.to(device)
            preds.append(model(xb).cpu().numpy())
            trues.append(yb.cpu().numpy())
    return np.vstack(preds), np.vstack(trues)

def train_one(model, train_dl, val_dl, lr=1e-3, wd=1e-4, max_epochs=40, patience=8, verbose=True):
    criterion = nn.SmoothL1Loss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3)

    best = float("inf"); best_state = None; wait = 0; best_epoch = 0
    for ep in range(1, max_epochs+1):
        model.train()
        for xb, yb in train_dl:
            xb = xb.to(device); yb = yb.to(device)
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
            optimizer.step()

        # validate
        vp, vt = eval_loader(model, val_dl)
        val_mae = mean_absolute_error(vt, vp)
        scheduler.step(val_mae)
        if verbose:
            print(f"Epoch {ep:02d} | Val MAE: {val_mae:.4f}")

        if val_mae < best - 1e-4:
            best = val_mae; wait = 0; best_epoch = ep
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
        else:
            wait += 1
            if wait >= patience:
                if verbose: print("Early stopping.")
                break

    if best_state:
        model.load_state_dict(best_state)
    return best, best_epoch


In [5]:

# ------------------- OPTUNA: tune key LSTM hyperparams -------------------
def objective(trial):
    hidden_size = trial.suggest_categorical("hidden_size", [96, 128, 160, 192, 256])
    num_layers  = trial.suggest_int("num_layers", 1, 3)
    dropout     = trial.suggest_float("dropout", 0.1, 0.4)
    lr          = trial.suggest_float("lr", 5e-4, 2e-3, log=True)
    batch_size  = trial.suggest_categorical("batch_size", [64, 128, 256])
    bidir       = trial.suggest_categorical("bidirectional", [False, True])

    train_dl, val_dl = make_loaders(X_train, y_train, X_val, y_val, batch_size)
    model = LSTMForecast(
        input_size=F,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=dropout,
        horizon=H,
        bidirectional=bidir
    ).to(device)

    best_mae, _ = train_one(
        model, train_dl, val_dl,
        lr=lr, wd=WD, max_epochs=EPOCHS, patience=PATIENCE, verbose=False
    )
    trial.report(best_mae, 1)
    return best_mae

sampler = optuna.samplers.TPESampler(seed=RANDOM_SEED)
pruner  = optuna.pruners.MedianPruner(n_warmup_steps=5)
study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)
print("\n=== Running Optuna hyperparameter search ===")
study.optimize(objective, n_trials=25, show_progress_bar=True)
print("\nBest trial:", study.best_trial.number)
print("Best value (Val MAE):", study.best_value)
print("Best params:", study.best_params)

# Extract best params (fall back to defaults if any missing)
best_params = study.best_params
HIDDEN_SIZE   = best_params.get("hidden_size", HIDDEN_SIZE)
NUM_LAYERS    = best_params.get("num_layers", NUM_LAYERS)
DROPOUT       = best_params.get("dropout", DROPOUT)
LR            = best_params.get("lr", LR)
BATCH_SIZE    = best_params.get("batch_size", BATCH_SIZE)
BIDIRECTIONAL = best_params.get("bidirectional", BIDIRECTIONAL)

# ------------------- Retrain best LSTM on the same split, then evaluate/save -------------------
print("\n=== Retraining best LSTM with Optuna params ===")
train_dl, val_dl = make_loaders(X_train, y_train, X_val, y_val, BATCH_SIZE)
model = LSTMForecast(F, HIDDEN_SIZE, NUM_LAYERS, DROPOUT, H, bidirectional=BIDIRECTIONAL).to(device)
best_val_mae, best_epoch = train_one(
    model, train_dl, val_dl, lr=LR, wd=WD, max_epochs=EPOCHS, patience=PATIENCE, verbose=True
)
print(f"Best epoch used: {best_epoch} | Best Val MAE saved: {best_val_mae:.4f}")

# Final metrics (Train + Val)
train_pred, train_true = eval_loader(model, train_dl)
val_pred,   val_true   = eval_loader(model, val_dl)

def metrics_block(y_t, y_p, label):
    mae  = mean_absolute_error(y_t, y_p)
    rmse = mean_squared_error(y_t, y_p, squared=False)
    r2s  = [r2_score(y_t[:, h], y_p[:, h]) for h in range(H)]
    print(f"\n=== {label} Metrics (averaged over {H} horizons) ===")
    print(f"MAE:  {mae:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"R²:   {np.mean(r2s):.3f}")
    mae_list  = [mean_absolute_error(y_t[:, h], y_p[:, h]) for h in range(H)]
    if H >= 12: print(f"First 12 horizons MAE: {[round(m,3) for m in mae_list[:12]]}")
    if H >= 24: print(f"24h MAE: {mae_list[23]:.3f}")
    if H >= 48: print(f"48h MAE: {mae_list[47]:.3f}")
    if H >= 72: print(f"72h MAE: {mae_list[71]:.3f}")
    return mae_list, r2s

train_mae_list, train_r2_list = metrics_block(train_true, train_pred, "Train")
val_mae_list,   val_r2_list   = metrics_block(val_true,   val_pred,   "Validation")

# Save per-horizon validation metrics
pd.DataFrame({
    "horizon": np.arange(1, H+1),
    "MAE": val_mae_list,
    "R2":  val_r2_list
}).to_csv(OUT_DIR / "lstm_val_per_horizon_metrics.csv", index=False)


[I 2025-08-12 01:01:02,459] A new study created in memory with name: no-name-5d9c97db-52fa-4ee3-9fb3-7d7c8ab41eac



=== Running Optuna hyperparameter search ===


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2025-08-12 01:01:29,656] Trial 0 finished with value: 4.446578025817871 and parameters: {'hidden_size': 128, 'num_layers': 1, 'dropout': 0.11742508365045984, 'lr': 0.0016613456824808538, 'batch_size': 128, 'bidirectional': False}. Best is trial 0 with value: 4.446578025817871.
[I 2025-08-12 01:17:09,710] Trial 1 finished with value: 4.546783447265625 and parameters: {'hidden_size': 256, 'num_layers': 2, 'dropout': 0.1873687420594126, 'lr': 0.0011677292338861146, 'batch_size': 256, 'bidirectional': True}. Best is trial 0 with value: 4.446578025817871.
[I 2025-08-12 01:17:44,448] Trial 2 finished with value: 4.948182106018066 and parameters: {'hidden_size': 256, 'num_layers': 1, 'dropout': 0.11951547789558387, 'lr': 0.0018631851866771777, 'batch_size': 64, 'bidirectional': True}. Best is trial 0 with value: 4.446578025817871.
[I 2025-08-12 01:18:04,964] Trial 3 finished with value: 4.6471171379089355 and parameters: {'hidden_size': 256, 'num_layers': 1, 'dropout': 0.2987566853061946, 

In [6]:

# ------------------- FORECAST NEXT 72 HOURS (ONE CSV) -------------------
# last window of *scaled* features (after cleaning)
last_window = df[feat_cols].values[-WINDOW_SIZE:]              # (T, F)
last_window_t = torch.tensor(last_window, dtype=torch.float32).unsqueeze(0).to(device)  # (1, T, F)

model.eval()
with torch.no_grad():
    future_pred = model(last_window_t).cpu().numpy().reshape(-1)   # (72,)

# anchor = last non-NaT time; start = +1 hour
last_valid_time = df.loc[df[TIME_COL].notna(), TIME_COL].iloc[-1]
start = last_valid_time.floor("H") + pd.Timedelta(hours=1)
future_times = pd.date_range(start=start, periods=PREDICT_HORIZON, freq="h")

# format as d/m/yy HH:MM (so 9/8/25 = 9 Aug 2025)
ft = pd.Series(future_times)
formatted_dt = (
    ft.dt.day.astype(str) + "/" +
    ft.dt.month.astype(str) + "/" +
    ft.dt.strftime("%y") + " " +
    ft.dt.strftime("%H:%M")
)

forecast_df = pd.DataFrame({
    "datetime": formatted_dt,
    "predicted_aqi_us": future_pred
})
forecast_df.to_csv(OUTPUT_FILE, index=False)
print(f"\nSaved forecast → {OUTPUT_FILE}")
print("First/last timestamps:", forecast_df['datetime'].iloc[0], "→", forecast_df['datetime'].iloc[-1])



Saved forecast → d:\Desktop\AlinasPrograms\myenv\10Pearls2\predictions\lstm_predicted_aqi_72hrs.csv
First/last timestamps: 10/8/25 00:00 → 12/8/25 23:00


In [7]:

# ------------------- SAVE MODEL + METADATA + SCALER -------------------
SAVE_DIR = PROJECT_ROOT / "models" / "current"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

MODEL_PATH   = SAVE_DIR / "lstm_multioutput_72h.pt"
SCALER_PATH  = SAVE_DIR / "scaler_lstm.joblib"
META_PATH    = SAVE_DIR / "metadata_lstm.json"   # <— separate file to avoid overwriting LGBM metadata

torch.save(model.state_dict(), MODEL_PATH)
joblib.dump(scaler, SCALER_PATH)

meta = {
    "features": feat_cols,                 # list of feature column names
    "target_col": TARGET_COL,              # "us_aqi"
    "time_col": TIME_COL,                  # e.g. "time" or "datetime"
    "window_size": WINDOW_SIZE,            # 24
    "horizon": PREDICT_HORIZON,            # 72
    "dayfirst": True,                      # parse like 4/8/25 -> 4 Aug
    "winsor_low": low,                     # 1% caps computed on train inputs
    "winsor_high": high,                   # 99% caps
    "train_medians": train_medians,        # feature medians (train inputs)
    "scaler_path": str(SCALER_PATH),       # saved scaler
    "optuna_best": study.best_params,      # tuned hyperparams
}
META_PATH.write_text(json.dumps(meta, indent=2))

print("Saved model   →", MODEL_PATH)
print("Saved scaler  →", SCALER_PATH)
print("Saved metadata→", META_PATH)


Saved model   → d:\Desktop\AlinasPrograms\myenv\10Pearls2\models\current\lstm_multioutput_72h.pt
Saved scaler  → d:\Desktop\AlinasPrograms\myenv\10Pearls2\models\current\scaler_lstm.joblib
Saved metadata→ d:\Desktop\AlinasPrograms\myenv\10Pearls2\models\current\metadata_lstm.json
